# Type Curve Analysis

This notebook demonstrates how to work with type curves and explore the impact of different parameters on production profiles.

## Overview

Type curves use the Arps decline equation to model oil/gas production over time:

- **Exponential decline** (b=0): $q(t) = q_i \cdot e^{-D_i t}$
- **Hyperbolic decline** (b>0): $q(t) = \frac{q_i}{(1 + b \cdot D_i \cdot t)^{1/b}}$

Where:
- $q_i$ = Initial production rate (bbl/d)
- $D_i$ = Initial decline rate (nominal, annual)
- $b$ = b-factor (0 = exponential, >0 = hyperbolic)
- $t$ = Time

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## Type Curve Functions

These functions mirror the TypeScript implementation in `utils/economics.ts`

In [ ]:
def calculate_type_curve(qi: float, b: float, di_annual: float, months: int = 120) -> np.ndarray:
    """
    Calculate monthly production rates using Arps decline.
    
    Parameters:
    - qi: Initial production rate (bbl/d)
    - b: b-factor (0 for exponential, >0 for hyperbolic)
    - di_annual: Nominal initial decline rate (annual, as decimal, e.g., 0.50 = 50%)
    - months: Number of months to project
    
    Returns:
    - Array of monthly production rates (bbl/month)
    """
    # Convert annual decline to monthly
    di_monthly = 1 - (1 - di_annual) ** (1/12)
    
    # Convert daily rate to monthly
    qi_monthly = qi * 30.4  # Average days per month
    
    rates = []
    
    for t in range(1, months + 1):
        if b == 0:
            # Exponential decline
            q_t = qi_monthly * np.exp(-di_monthly * t)
        else:
            # Hyperbolic decline
            q_t = qi_monthly / ((1 + b * di_monthly * t) ** (1/b))
        
        rates.append(q_t)
    
    return np.array(rates)

def calculate_eur(rates: np.ndarray) -> float:
    """Calculate Estimated Ultimate Recovery (EUR) in barrels."""
    return np.sum(rates)

## Example 1: Base Case Type Curve

In [ ]:
# Base case parameters
qi = 1000      # bbl/d
b = 1.2        # Hyperbolic
di = 0.50      # 50% annual decline
months = 120   # 10 years

# Calculate production profile
rates = calculate_type_curve(qi, b, di, months)
eur = calculate_eur(rates)

# Create DataFrame for easier plotting
df = pd.DataFrame({
    'Month': range(1, months + 1),
    'Production (bbl/month)': rates
})

# Plot
plt.figure(figsize=(12, 6))
plt.plot(df['Month'], df['Production (bbl/month)'], linewidth=2)
plt.title(f'Type Curve: qi={qi} bbl/d, b={b}, Di={di*100}% | EUR={eur:,.0f} bbl', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Production (bbl/month)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"EUR: {eur:,.0f} bbl")
print(f"Year 1 Production: {rates[:12].sum():,.0f} bbl")
print(f"Final Month Rate: {rates[-1]:,.0f} bbl/month ({rates[-1]/30.4:.1f} bbl/d)")

## Example 2: Impact of b-factor

Compare exponential (b=0) vs hyperbolic (b>0) decline

In [ ]:
# Compare different b-factors
b_values = [0, 0.5, 1.0, 1.5, 2.0]
colors = plt.cm.viridis(np.linspace(0, 1, len(b_values)))

plt.figure(figsize=(14, 7))

for b_val, color in zip(b_values, colors):
    rates = calculate_type_curve(qi=1000, b=b_val, di_annual=0.50, months=120)
    eur = calculate_eur(rates)
    label = f'b={b_val} (EUR={eur:,.0f} bbl)'
    if b_val == 0:
        label = f'b={b_val} - Exponential (EUR={eur:,.0f} bbl)'
    plt.plot(range(1, 121), rates, linewidth=2, label=label, color=color)

plt.title('Impact of b-factor on Production Profile (qi=1000 bbl/d, Di=50%)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Production (bbl/month)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 3: Impact of Initial Decline Rate

In [ ]:
# Compare different decline rates
di_values = [0.30, 0.40, 0.50, 0.60, 0.70]
colors = plt.cm.plasma(np.linspace(0, 1, len(di_values)))

plt.figure(figsize=(14, 7))

for di_val, color in zip(di_values, colors):
    rates = calculate_type_curve(qi=1000, b=1.2, di_annual=di_val, months=120)
    eur = calculate_eur(rates)
    label = f'Di={di_val*100:.0f}% (EUR={eur:,.0f} bbl)'
    plt.plot(range(1, 121), rates, linewidth=2, label=label, color=color)

plt.title('Impact of Initial Decline Rate on Production Profile (qi=1000 bbl/d, b=1.2)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Production (bbl/month)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example 4: Cumulative Production

Visualize how EUR accumulates over time

In [ ]:
# Calculate base case
rates = calculate_type_curve(qi=1000, b=1.2, di_annual=0.50, months=120)
cumulative = np.cumsum(rates)

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot monthly rates
ax1.plot(range(1, 121), rates, linewidth=2, color='steelblue')
ax1.set_title('Monthly Production Rate', fontsize=14)
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Production (bbl/month)', fontsize=12)
ax1.grid(True, alpha=0.3)

# Plot cumulative
ax2.plot(range(1, 121), cumulative, linewidth=2, color='coral')
ax2.set_title('Cumulative Production (EUR)', fontsize=14)
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Cumulative Production (bbl)', fontsize=12)
ax2.axhline(y=cumulative[-1], color='red', linestyle='--', alpha=0.5, label=f'Final EUR: {cumulative[-1]:,.0f} bbl')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print key statistics
print(f"EUR Milestones:")
print(f"  After 12 months: {cumulative[11]:,.0f} bbl ({cumulative[11]/cumulative[-1]*100:.1f}%)")
print(f"  After 24 months: {cumulative[23]:,.0f} bbl ({cumulative[23]/cumulative[-1]*100:.1f}%)")
print(f"  After 36 months: {cumulative[35]:,.0f} bbl ({cumulative[35]/cumulative[-1]*100:.1f}%)")
print(f"  After 60 months: {cumulative[59]:,.0f} bbl ({cumulative[59]/cumulative[-1]*100:.1f}%)")
print(f"  Final EUR: {cumulative[-1]:,.0f} bbl (100%)")

## Next Steps

- Integrate with backend Python economics module (once implemented)
- Add economic calculations (revenue, costs, NPV)
- Compare with TypeScript implementation for parity testing
- Explore sensitivity to different parameter combinations